# Present Value of Liabilities and Funding Ratio

The present value of a set of liabilities $L$ where each liability $L_i$, is due at time $t_i$ is given by:

$$PV(L) = \sum^k_{i=1} B(t_i)L_i$$

where $B(t_i)$ is the price of a pure discount bond that pays 1 dollar at time $t_i$.

If we assume the yield curve is flat and the annual rate of interest is $r$ then $B(t)$ is given by

$$B(t) = \frac{1}{(1+r)^t}$$

The goal of investing isn't only to grow your assets to as large a number as possible. The goal of investing is to make sure that you will have the resources to be able to have the consumption patterns that you want in the future.

We are going to try and come up with some basic functionality that allows us to measure two things.
1. What is this future liability, this future consumption, and how do we model it?
2. Do I have enough money today to be able to meet that, and how do I measure that?
    * This is what we call the funding ratio.

So we are going to come up with a formal, practical way of measuring liabilities that is useful for our investment purposes. And we are going to come up with a measure of how we're doing with the assets we have today, in order to meet those future liabilities, this is the funding ratio.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import edhec_risk_kit as erk

In [8]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time t,
    given interest rate r
    """
    return 1/(1+r)**t

In [11]:
# $0.74 is what it would cost to get $1 back at the end of 10 years
discount(10, 0.03)

0.7440939148967249

In [12]:
# verify this is the case
0.7440939148967249*(1.03**10)

1.0

In [16]:
def pv(l, r):
    """
    Compute the present value of a sequence of liabilities
    l is indexed by the time and the values are the amounts of each liability
    returns the present value of the sequence
    """
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

In [17]:
liabilities = pd.Series(data=[1, 1.5, 2, 2.5],
                        index=[3, 3.5, 4, 4.5])
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64

In [18]:
# The sum of the liabilities is $7M
# The present value of those liabilities because of the time-value of money is $6.23M
pv(liabilities, 0.03)

6.233320315080044

In [19]:
# Compute the funding ratio
# I owe $7M in the future, but the PV of $6.23M
# If I had the $6.23M today and this sequence of liabilities in the future
# I can pay it off because I know the interest rates are 3%

In [20]:
def funding_ratio(assets, liabilities, r):
    """
    Compute the funding ratio of some assets given liabilities and interest rate
    """
    return assets/pv(liabilities, r)

In [21]:
funding_ratio(5, liabilities, 0.03)

0.8021407126958778

In [22]:
# We are under funded, we are 80% funded.
# Unless we do something else with the $5M other than put it in bonds
# We can't expect to meet our liabilities

In [23]:
funding_ratio(5, liabilities, 0.02)

0.7720304366941647

In [24]:
# Our funding ratio is even worse
# The value of the liabilities have gone up now
# Put differently, the amount by which the assets that we have now will grow more slowly

In [25]:
funding_ratio(5, liabilities, 0.05)

0.8649082768407929

In [26]:
# Our funding ratio has gone up

In [27]:
import ipywidgets as widgets
from IPython.display import display

In [29]:
# What do interest rates and asset values do to our funding ratio
# How does our funding ratio change if either the assets I have increase
# or the interest rates change
def show_funding_ratio(assets, r):
    fr = funding_ratio(assets, liabilities, r)
    print(f"{fr*100:.2f}%")
    
controls = widgets.interactive(show_funding_ratio,
                               assets = widgets.IntSlider(min=1, max=10, step=1, value=5),
                               r = (0, .20, .01))

display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

In [30]:
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64